### <span style="color:teal"> __UNIPARTITE - DISEASE NETWORK CONSTRUCTION__ 

In [31]:
# LOADING REQUIRED PACKAGES
import numpy as np
import pickle
import json
import textwrap
import ipycytoscape
import io, base64
import networkx as nx
import matplotlib.pyplot as plt
from adjustText import adjust_text
from IPython.display import HTML, display
from ipycytoscape import CytoscapeWidget
from io import BytesIO

In [6]:
# LOADING DICTIONARY DISEASE NAMES AND DISEASE CATEGORIES

# Open the JSON file containing disease name mappings
with open("/mnt/iusers01/fatpou01/bmh01/msc-bioinf-2024-2025/h44063jg/gmp_jms/disease_dict.json", "r") as f:
    # Load the contents of the JSON file into a Python dictionary
    disease_dict = json.load(f)
    
# Open hte JSOn file contaiing the disease classificaiton data
with open("/mnt/iusers01/fatpou01/bmh01/msc-bioinf-2024-2025/h44063jg/gmp_jms/disease_classification.json", "r") as f:
    disease_classification = json.load(f)

# Open the JSON file containing taxon name mappings
with open("/mnt/iusers01/fatpou01/bmh01/msc-bioinf-2024-2025/h44063jg/gmp_jms/taxon_dict.json", "r") as f:
    # Load the contents of the JSON file into a Python dictionary
    taxon_dict = json.load(f)
    
# Open the JSON file containing taxon phyla classification name mappings
with open("/mnt/iusers01/fatpou01/bmh01/msc-bioinf-2024-2025/h44063jg/gmp_jms/taxon_phyla_dict.json", "r") as f:
    # Load the contents of the JSON file into a Python dictionary
    taxon_phyla_dict = json.load(f)

In [7]:
# LOADING THE UNIPARTITE DISEASE _GENUS PROJECTION NETWORK
with open("/mnt/iusers01/fatpou01/bmh01/msc-bioinf-2024-2025/h44063jg/gmp_jms/unipartite_disease_genus.gpickle", "rb") as f:
    G_disease = pickle.load(f)

In [42]:
# Threshold at weight >= 0 @80%
theta = 8
G_disease_thresholded = G_disease.edge_subgraph(
    [(u, v) for u, v, d in G_disease.edges(data=True) if d.get("weight", 0) >= theta]
).copy()

# First compute degree for each node and add as attribute
degree_dict = dict(G_disease_thresholded.degree())
nx.set_node_attributes(G_disease_thresholded, degree_dict, "degree")

# Add 'label' attribute from disease_dict
nx.set_node_attributes(G_disease_thresholded, disease_dict, "label")

# Create the Cytoscape widget
cyto_widget = CytoscapeWidget()
cyto_widget.graph.add_graph_from_networkx(
    G_disease_thresholded,
    directed=False  # change to True if your network is directed
)

# Increase canvas size
cyto_widget.layout.width = '1200px'
cyto_widget.layout.height = '900px'

# --- Style ---
cyto_widget.set_style([{
    'selector': 'node',
    'style': {
        'label': 'data(label)',   # <-- use disease name here
        'background-color': '#1f77b4',
        'width': 'mapData(degree, 1, {}, 20, 80)'.format(max(degree_dict.values())),
        'height': 'mapData(degree, 1, {}, 20, 80)'.format(max(degree_dict.values())),
        'color': 'black',
        'font-size': '10px',
        'text-valign': 'center'
    }
}, {
    'selector': 'edge',
    'style': {
        'line-color': 'light grey',
        'width': 'mapData(weight, {}, {}, 1, 6)'.format(
            min([d.get('weight',0) for u,v,d in G_disease_thresholded.edges(data=True)]),
            max([d.get('weight',0) for u,v,d in G_disease_thresholded.edges(data=True)])
        )
        }
    }
])


# --- Concentric layout ---
cyto_widget.set_layout(
    name='concentric',
    concentric='function(node){return node.data("degree");}',  # hub score = degree
    levelWidth=1.5,       # spacing between rings
    minNodeSpacing=40,    # more breathing room between nodes
    startAngle=3.14/2     # optional: rotate layout
)


cyto_widget

CytoscapeWidget(cytoscape_layout={'name': 'concentric', 'concentric': 'function(node){return node.data("degree…

In [41]:
import networkx as nx
import matplotlib.pyplot as plt
from ipycytoscape import CytoscapeWidget
from io import BytesIO
import base64
import numpy as np

# -----------------------------
# Threshold edges
theta = 8
G_disease_thresholded = G_disease.edge_subgraph(
    [(u, v) for u, v, d in G_disease.edges(data=True) if d.get("weight", 0) >= theta]
).copy()

# -----------------------------
# Node attributes
degree_dict = dict(G_disease_thresholded.degree())
nx.set_node_attributes(G_disease_thresholded, degree_dict, "degree")
nx.set_node_attributes(G_disease_thresholded, disease_dict, "label")  # disease names
nx.set_node_attributes(G_disease_thresholded, disease_classification, "categories")  # multi-category list

# -----------------------------
# Cytoscape widget
cyto_widget = CytoscapeWidget()
cyto_widget.graph.add_graph_from_networkx(G_disease_thresholded, directed=False)

# -----------------------------
# Prepare colors for categories
categories = list({cat for cats in disease_classification.values() for cat in cats})
palette = plt.cm.tab20(np.linspace(0, 1, len(categories)))
category_color_map = {cat: palette[i] for i, cat in enumerate(categories)}

# -----------------------------
# Generate pie-chart images for nodes
node_images = {}
for node_id, cats in disease_classification.items():
    counts = [1 if c in cats else 0 for c in categories]
    if sum(counts) == 0:
        counts = [1]
    fig, ax = plt.subplots(figsize=(0.4,0.4))
    slice_colors = [category_color_map[c] for c, val in zip(categories, counts) if val]
    ax.pie([v for v in counts if v], colors=slice_colors)
    ax.axis('equal')
    plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format='png', dpi=300, transparent=True)
    plt.close(fig)
    img_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
    node_images[node_id] = "data:image/png;base64," + img_base64

# Assign images to Cytoscape nodes
for node in cyto_widget.graph.nodes:
    node_id = node.data["id"]
    if node_id in node_images:
        node.data["background-image"] = node_images[node_id]
        node.data["background-fit"] = "cover"
        node.data["background-color"] = "transparent"

# -----------------------------
# Increase canvas size
cyto_widget.layout.width = '1200px'
cyto_widget.layout.height = '900px'

# -----------------------------
# Style nodes and edges
cyto_widget.set_style([
    {
        'selector': 'node',
        'style': {
            'label': 'data(label)',
            'width': 'mapData(degree, 1, {}, 20, 80)'.format(max(degree_dict.values())),
            'height': 'mapData(degree, 1, {}, 20, 80)'.format(max(degree_dict.values())),
            'color': 'black',
            'font-size': '10px',
            'text-valign': 'center'
        }
    },
    {
        'selector': 'edge',
        'style': {
            'line-color': 'lightgreen',  # very light green
            'width': 'mapData(weight, {}, {}, 1, 6)'.format(
                min([d.get('weight',0) for u,v,d in G_disease_thresholded.edges(data=True)]),
                max([d.get('weight',0) for u,v,d in G_disease_thresholded.edges(data=True)])
            )
        }
    }
])

# -----------------------------
# Concentric layout (high-degree nodes at center)
cyto_widget.set_layout(
    name='concentric',
    concentric='function(node){return node.data("degree");}',
    levelWidth=1.5,
    minNodeSpacing=40,
    startAngle=3.14/2
)

# -----------------------------
# Display the network
cyto_widget


CytoscapeWidget(cytoscape_layout={'name': 'concentric', 'concentric': 'function(node){return node.data("degree…